In [ ]:
import pretty_midi
import numpy as np
import muspy
import os
import csv
%run ~/violin-renderer/src/misc/parse.ipynb

In [ ]:
# Install soundfont for muspy
muspy.download_musescore_soundfont()

In [ ]:
# @param:
    # startTimings: an array of expressive start times
    # endTimings: an array of expressive end times
    # fileName: name of CSV file
# @return: output a CSV file containing the given expressive start and end timings
def storeCSV(startTimings, endTimings, fileName):
    if len(startTimings) != len(endTimings):
        raise ValueError("Arrays must have the same length")
    else:
        rows = zip(startTimings, endTimings)
        outputDirectory = './csv-output/'
        with open(outputDirectory + fileName + '.csv', mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['start', 'end'])  # Write header
            writer.writerows(rows)

In [ ]:
# @param:
    # inputPath: path to the input file
    # tempoPath: path to the tempo file
    # outputPath: path to store file
    # name: name of the .wav file
# @return: wav file in folder that converts the expressive timings
def synthesizeSourceInput(inputPath, tempoPath, outputPath, name):
    MIDI = pretty_midi.PrettyMIDI()

    # Create an Instrument instance for a violin instrument
    violinProgram = pretty_midi.instrument_name_to_program('Violin')
    violin = pretty_midi.Instrument(program=violinProgram)

    # Create notes
    inputInfo = parseInputFile(inputPath)
    tempo = parseTempoFile(tempoPath)    

    # Add notes to instrument
    for notes in inputInfo:
        # We convert from tick to actual timings in seconds
        start = (notes[0] / 24) / tempo * 60
        end = (notes[1] / 24) / tempo * 60
        note = pretty_midi.Note(
            velocity=64, pitch=int(notes[2]), start=start, end=end)
        violin.notes.append(note)

    # Add the instrument to the PrettyMIDI object
    MIDI.instruments.append(violin)

    # Write to a MIDI file
    MIDI.write('violin.mid')

    # Convert to WAV using fluidsynth
    soundfontPath = os.path.expanduser("~") + '/.muspy/musescore-general/MuseScore_General.sf3'
    os.system(f'fluidsynth -ni {soundfontPath} {"violin.mid"} -F {outputPath + name + ".wav"} -r 44100')

    # Remove the MIDI file
    os.remove('violin.mid')

In [ ]:
# @param:
    # notesPath: path to the original input file -> to get pitch and velocity
    # timingsPath: path to the generative output file containing the expressive start and end timings
    # outputPath: path to store file
    # name: name of the .wav file
# @return: wav file in src folder that converts the expressive timings
def synthesizeOutput(notesPath, timingsPath, outputPath, name):
    MIDI = pretty_midi.PrettyMIDI()

    # Create an Instrument instance for a violin instrument
    violinProgram = pretty_midi.instrument_name_to_program('Violin')
    violin = pretty_midi.Instrument(program=violinProgram)

    # Create notes
    notesInfo = parseInputFile(notesPath)
    timingsInfo = parseOutputFile(timingsPath)

    for notes, times in zip(notesInfo, timingsInfo):
        note = pretty_midi.Note(
            velocity=64, pitch=int(notes[2]), start=times[0], end=times[1])
        violin.notes.append(note)

    # Add the instrument to the PrettyMIDI object
    MIDI.instruments.append(violin)

    # Write to a MIDI file
    MIDI.write('violin.mid')

    # Convert to WAV using fluidsynth
    soundfontPath = os.path.expanduser("~") + '/.muspy/musescore-general/MuseScore_General.sf3'
    os.system(f'fluidsynth -ni {soundfontPath} {"violin.mid"} -F {outputPath + name + ".wav"} -r 44100')

    # Remove the MIDI file
    os.remove('violin.mid')